In [1]:
#%matplotlib inline
import numpy as np
import pandas as pd
import scipy.sparse
from sklearn import preprocessing
from sklearn.grid_search import ParameterGrid
import xgboost as xgb
import pickle
#import graphviz
import matplotlib.pyplot as plt
%matplotlib inline

Read in just the testing and training data.

In [3]:
error = pd.read_pickle('scripts/cv_results/actions_e20/search3/res1.p')
error2 = pd.read_pickle('scripts/cv_results/actions_e20/search3/res5.p')
error3 = pd.read_pickle('scripts/cv_results/actions_e20/search3/res4.p')
e = pd.read_pickle('scripts/cv_results/actions_e20/errors_search3.p')
param = pd.read_pickle('scripts/cv_results/actions_e20/params_search3.p')
#error['mean'] = error.apply(np.mean, axis = 1)
# error = error.astype(np.float64)
# error['mean'] = error.mean(1)
# error['std'] = error.std(1)

In [4]:
e

,train-error-mean0,train-error-std0,valid-error-mean0,valid-error-std0,train-error-mean1,train-error-std1,valid-error-mean1,valid-error-std1,train-error-mean2,train-error-std2,...,valid-error-mean3,valid-error-std3,train-error-mean4,train-error-std4,valid-error-mean4,valid-error-std4,train-error-mean5,train-error-std5,valid-error-mean5,valid-error-std5
0,0.178379,0.004976,0.152968,0.006064,0.173809,0.004826,0.150643,0.005860,0.170264,0.002849,...,0.150647,0.005864,0.172937,0.006202,0.150405,0.006636,0.170337,0.003050,0.148132,0.004279
1,0.178120,0.004385,0.151890,0.004548,0.170603,0.004145,0.147612,0.005687,0.168471,0.002560,...,0.147595,0.005676,0.171518,0.005253,0.148438,0.005285,0.168715,0.002530,0.146128,0.003183
2,0.176438,0.003856,0.150462,0.004155,0.169283,0.003318,0.145850,0.005410,0.166753,0.002069,...,0.145862,0.005474,0.170669,0.002909,0.148152,0.003687,0.166994,0.002222,0.144785,0.003984
3,0.175972,0.004285,0.150457,0.004517,0.168220,0.003000,0.145520,0.005280,0.166335,0.001976,...,0.145590,0.005300,0.170320,0.003287,0.147170,0.004454,0.166547,0.002078,0.144565,0.003740
4,0.174973,0.003193,0.149943,0.003739,0.166420,0.001812,0.144578,0.004554,0.165406,0.000695,...,0.144728,0.004611,0.170182,0.002675,0.147348,0.004040,0.165870,0.001081,0.144008,0.003589
5,0.173802,0.002891,0.149322,0.004237,0.165916,0.001488,0.144415,0.004366,0.165476,0.000952,...,0.144413,0.004370,0.170002,0.002408,0.146948,0.003831,0.166065,0.001690,0.144043,0.003730
6,0.173025,0.003237,0.149010,0.004467,0.165549,0.001292,0.144208,0.004338,0.164918,0.000550,...,0.144238,0.004474,0.169468,0.002624,0.146682,0.003733,0.165409,0.000821,0.143648,0.003336
7,0.171984,0.002998,0.148662,0.004213,0.165094,0.001012,0.143623,0.004576,0.164560,0.000446,...,0.143677,0.004699,0.169131,0.002495,0.146403,0.003612,0.164909,0.000601,0.143262,0.003754
8,0.171740,0.002954,0.148642,0.003926,0.164963,0.001084,0.143688,0.004774,0.164245,0.000467,...,0.143858,0.004732,0.168528,0.002087,0.145753,0.003566,0.164656,0.000530,0.143085,0.003617
9,0.171083,0.002925,0.148203,0.003843,0.164605,0.000817,0.143662,0.004706,0.164109,0.000518,...,0.143665,0.004733,0.168338,0.002072,0.145893,0.003455,0.164486,0.000593,0.142937,0.003714


In [150]:
pred = a.argsort(axis = 1)[:,::-1][:,0:5]

In [147]:
a.argsort(axis = 1)[:,::-1]

array([[ 0,  7,  4, 11,  3,  2,  6,  1, 10,  5,  9,  8],
       [10,  5,  0,  7, 11,  3,  2,  6,  9,  1,  8,  4],
       [ 9,  8,  1,  2, 10,  4, 11,  7,  0,  6,  5,  3],
       [ 6,  2,  3,  1,  9,  0, 11, 10,  7,  8,  4,  5],
       [ 5,  3,  7,  6,  2,  8,  9,  0,  4, 11, 10,  1],
       [ 6,  5, 11,  8,  1, 10,  0,  7,  4,  2,  3,  9],
       [ 2, 11,  0,  8,  9,  1,  7,  6,  4,  3, 10,  5],
       [ 5,  4,  7,  0,  3, 11, 10,  8,  9,  6,  2,  1],
       [ 8,  9, 11,  0,  2,  1,  7,  4,  6, 10,  5,  3],
       [ 5,  4, 11,  9,  0,  3,  7,  8,  2,  1, 10,  6]])

In [152]:
sub = pd.read_csv('submissions/actions2_e16_53n.csv')

In [162]:
sub[4::5].country.value_counts()

IT       26954
FR       14845
GB        9558
ES        7756
CA        1988
DE         336
NL         301
other      169
AU         115
PT          47
NDF         24
US           3
dtype: int64

In [138]:
label = np.array([ 7,  7, 10, 7,  10,  7 ,8])

In [164]:
pred = pred[:,1]

In [165]:
pred

array([ 7,  5,  8,  2,  3,  5, 11,  4,  9,  4])

In [173]:
np.logical_or(label == 7, label == 8)

array([ True,  True, False,  True, False,  True,  True], dtype=bool)

In [108]:
idx = np.where(label == 10)[0]

In [109]:
b[idx]

array([10,  5])

In [110]:
idx

array([2, 4])

In [111]:
len(idx)

2

In [121]:
np.sum(np.equal(b[idx], 10)) / len(idx)

0.5

In [116]:
np.sum(np.equal(b[idx], 10))

1

In [75]:
b = a.argsort(axis = 1)[:,::-1][:,0]

In [22]:
error['mean'] = error.astype(np.float).mean(axis = 1)

In [3]:
label = np.array([1, 2, 3, 1])
np.where(label == 1)

(array([0, 3]),)

In [23]:
param.to_pickle('scripts/cv_results/actions_e20/params_search3.p')

In [ ]:
len()

In [31]:
error3.iloc[-10:]

,0,1,2,3,4,5
90,0.13536,0.14114,0.13912,0.14204,0.14541,0.13829
91,0.13561,0.14124,0.13916,0.14229,0.14555,0.13822
92,0.13558,0.14122,0.13897,0.14225,0.14555,0.13823
93,0.13551,0.14108,0.13876,0.14223,0.14543,0.13803
94,0.13577,0.14111,0.13889,0.14204,0.14559,0.13800
95,0.13570,0.14102,0.13890,0.14200,0.14537,0.13783
96,0.13560,0.14116,0.13876,0.14189,0.14541,0.13804
97,0.13552,0.14127,0.13894,0.14217,0.14551,0.13813
98,0.13560,0.14140,0.13896,0.14210,0.14547,0.13799
99,0.13562,0.14131,0.13898,0.14212,0.14522,0.13809


In [33]:
for i in range(6):
    print np.argmin(error2.iloc[-10:, i])

98
98
90
91
99
98


In [32]:
e.loc[90:,e.columns.str.contains('valid.*mean')]

,valid-error-mean0,valid-error-mean1,valid-error-mean2,valid-error-mean3,valid-error-mean4,valid-error-mean5
90,0.140602,0.140572,NaN,NaN,0.140227,0.140545
91,0.140607,0.140480,NaN,NaN,0.140345,0.140532
92,0.140598,0.140462,NaN,NaN,0.140300,0.140552
93,0.140698,0.140447,NaN,NaN,0.140173,0.140488
94,0.140628,0.140388,NaN,NaN,0.140233,0.140462
95,0.140587,0.140428,NaN,NaN,0.140137,0.140513
96,0.140610,0.140410,NaN,NaN,0.140143,0.140518
97,0.140653,0.140438,NaN,NaN,0.140257,0.140582
98,0.140640,0.140422,NaN,NaN,0.140253,0.140462
99,0.140680,0.140645,NaN,NaN,0.140223,0.140515


In [34]:
param

,subsample,eta,colsample_bytree,max_depth,details
0,0.7,0.16,0.30,6,actions2
1,0.7,0.16,0.60,6,actions2
2,0.7,0.16,0.60,6,actions3_nodrop
3,0.7,0.16,0.60,6,actions3_dropped_p4total
4,0.9,0.13,0.45,6,actions3_nodrop
5,0.9,0.13,0.60,6,actions3_nodrop


In [3]:
#train_100 = pd.read_csv('Airbnb_data/train_users_2.csv', nrows = 100, index_col='id')
train_data = pd.read_csv('data/train_users_2.csv', index_col = 'id')
test_data = pd.read_csv('data/test_users.csv', index_col = 'id')
#sessions = pd.read_csv('../data/sessions.csv')
#age = pd.read_csv('Airbnb_data/age_gender_bkts.csv')
#countries = pd.read_csv('Airbnb_data/countries.csv')
all_df = pd.read_pickle('data/all_df.p')

In [163]:
for i in np.unique(test_data.first_browser):
    if i not in np.unique(train_data.first_browser):
        print i

IBrowse
Nintendo Browser
UC Browser


In [5]:
error = pd.read_pickle('scripts/cv_results.p')
params = pd.read_pickle('scripts/cv_params.p')

In [6]:
res = pd.concat([error, params], axis = 1)

In [7]:
res.sort_values(by = 'test-error-mean', axis = 0);

In [3]:
all_df = feakaggle_model(all_df)

#one hot encoding of features
print 'number of columns before one hot encoding', all_df.shape[1]
ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
all_df = pd.get_dummies(all_df, columns = ohe_feats)
print 'number of columns after one hot encoding', all_df.shape[1]

number of columns before one hot encoding 17
number of columns after one hot encoding 161


In [6]:
with open('xgbmodels/sessions_e20_25n.p', 'rb') as f:
    bst = pickle.load(f)
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, all_df.columns[int(feat[1:])], score) for feat, score in features]

NameError: name 'all_df' is not defined

In [ ]:
xgb.to_graphviz(bst, num_trees=11)

In [ ]:
submission = pd.DataFrame(np.column_stack([idx, cntr]), columns = ['id', 'country'])
submission.to_csv('submissions/datebin_model.csv', index=False)